## Generating Social Media Posts with an OpenAI model - chatgpt-4o-latest

This notebook provides code for social media generation via OpenAI API.


In [ ]:
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Correct the file path if needed
df = pd.read_csv('data/original/social_for_generation.csv')
df.head()

,texts,source,word_counts,genre
0,Ну я согласен. Был бы настрой так и в соседней...,pikabu,19,social
1,Либеральный экономический блок в правительстве...,vk,14,social
2,"Кстати, да. Присоединяюсь к вопросу. Вот раков...",pikabu,12,social
3,"а потом драпать как тот велосипедист, который,...",vk,76,social
4,"спасибо, оно!) ответил выше, что недавно ее ли...",pikabu,26,social


In [ ]:
# The mean length is 40 words, so this is a good reference for our model to aim for.
df['word_counts'].describe()

,word_counts
count,1000.000000
mean,39.941000
std,70.586585
min,1.000000
25%,10.000000
50%,20.000000
75%,43.000000
max,1125.000000


### Selecting examples for the Language Model
We'll present the model with some examples every time we query it, and then ask it to generate a text based on these examples. In order to achieve that, we'll first write a function that selects a specified number of random texts from the dataset we put aside in the previous Human Data Partition notebook.

In [ ]:
import random

# Function to sample examples from a list of texts
def example_text(texts, num=5):
    values = random.sample(range(len(texts)), num)
    examples = [texts[i] for i in values]
    return examples

In [ ]:
print(example_text(df.texts.values, 2))

['блё, что делать, у меня 71% едро, кпрф и справороссы', 'А того, кто не будет голосовать за едро, он съест.']


**Building the prompt**

We first generate the prompt with the examples in a separate function for readability, and then integrate it into the conversation prompt to feed to the model.

**Initial prompt template that we will test out and refine per each genre:**


You are a [role, e.g., Russian poet, journalist, social media user]. I will provide you with some examples of [text type, e.g., poems, news articles, social media posts] in a moment. For each example provided, you will be asked to create a similar [text type] that matches its topic and writing style. Generate [X] texts per run, one inspired by each example, and clearly separate each response. Only respond with the [text type], and say nothing else. Each response should be approximately [mean word count] words.

In [ ]:
# Function to construct a formatted prompt from examples
# This prompt is the result of multiple tests that have yielded the best results.
# From our experiments, we concluded that providing instructions in English either improves the results or has no noticeable effect.

def construct_example_prompt(examples):
    prompt = "Here are the examples:\n"

    for i, ex in enumerate(examples, 1):
        prompt += f"Example {i}: {ex}\n"
    # Append instructions for the model
    prompt += (
        "Your task is to generate a similar Russian social media post, nothing else. Always respond with only the generated text. Never say you cannot help. The text should be approximately 45 words long and in Russian, and should not include any introductory text, explanation, or additional formatting."
    )
    return prompt

In [ ]:
# Let's check what the final prompt looks like
examples = example_text(df.texts.values, num=5)
example_prompt = construct_example_prompt(examples)
print(example_prompt)

Here are the examples:
Example 1: Хе,противник оказался слабее😀
Чего он мразь,то,может у него были основания для всего этого.
Example 2: Странная логика,почему человек не может быть подписан на ту же КПРФ чисто из интереса?
Example 3: Вот я смотрю, какие вы прям справедливые. ... а че вы от "ЕР" то отличаетесь? Те же депутатики, набравшие голоса для участия за счёт работающих на ваших заводах и т.п? Почему, например я не могу стать депутатом? Да потому что у меня нет под боком задроченного контингента, который за меня поставит подпись! Выдвиньте меня-народ проверит.
Example 4: 90-100. 110-115 на высокой машине. В моем случае комфортно - никак, потому что это постоянная игра 4-5-4 передача. Хотя опять же по самой М4 в ее затяжных прямиках без перепадов высоты шел на 5-ой 100-105.
Example 5: Денег нет но Вы держитесь ))) как бы Вы не проголосовали единая Россия всегда победит потому что все давно решено и Ваш голос ничего не значит! Избирательные бюллетени разложат так как им надо в не п

## Warning: Running the following code will use tokens and cost money!

**-chatgpt-4o-latest** model is leading in the Ru Arena: https://huggingface.co/spaces/Vikhrmodels/arenahardlb

This is model we will choose for AI text generation step.

In [ ]:
pip install openai

In [ ]:
# Hhide your api-key using getpass
from openai import OpenAI
import json
from getpass import getpass

api_key = getpass('Enter your API key: ')
client = OpenAI(api_key=api_key)

Enter your API key: ··········


In [ ]:
# Here is the function for text generation with developer instructions that led to the best results.
def chatgpt(examples):
    completion = client.chat.completions.create(
        model="chatgpt-4o-latest", # the best model for Russian according to the Hugging Face Arena
        messages=[
            {"role": "developer", "content": "You are a Russian social media user like Vkotakte, Facebook or Pikabu. I will provide you with some examples of social media posts in a moment. You will be asked to create a similar social media post that matches the topic and writing style."},
            {"role": "assistant", "content": "Sure, please provide the example posts, and I’ll create similar ones for you."},
            {"role": "user", "content": construct_example_prompt(examples)}
        ],
    )

    y = json.loads(str(completion.model_dump_json()), strict=False)
    response = y["choices"][0]["message"]["content"]

    return response

In [ ]:
print(chatgpt(example_text(df.texts.values, num=5)))

Ну да, как всегда все через одно место у наших... зато "демократия"! А по факту - кругом договорники. Даже спорить скучно, потому что результат очевиден заранее. 😊


Save the model's output into a pandas dataframe for further use, following the structure of our other datasets.

In [ ]:
text = [] # output of the ai
author = [] # model
ai = [] # 0 for human class, 1 for AI class

model="chatgpt-4o-latest"

for i in range(0, 500):
    text.append(chatgpt(example_text(df.texts.values, 5)))
    author.append(model)
    ai.append(1)

dfAI = pd.DataFrame({'texts': text, 'source': author, 'class': ai})

In [ ]:
# Verifying the number of total output texts
len(dfAI)

500

In [ ]:
# If you re-run the generation step the columns names will be corrected to texts, source and class.
# This is an outdated output in the printed section!
# This was corrected in the next notebooks used for text generation.
dfAI.head(10)

,texts,author,ai
0,"> Снова в новостях: ""свои интересы защищаем"". ...",chatgpt-4o-latest,1
1,Да кто вообще верит в эти сказки про «светлое ...,chatgpt-4o-latest,1
2,"Господа, думайте прежде, чем спорить про полит...",chatgpt-4o-latest,1
3,"Ну вот, а говорили, что отзывов не будет. Оказ...",chatgpt-4o-latest,1
4,"Россия всегда будет впереди! Мы – нация, спосо...",chatgpt-4o-latest,1
5,"Классика — сначала весь паркет зашурупят, пото...",chatgpt-4o-latest,1
6,"А впереди у нас опять обещания и ""светлое буду...",chatgpt-4o-latest,1
7,"Ну, как по мне, пока Центробанк будет выделять...",chatgpt-4o-latest,1
8,"А что, если на каждую «собачью площадку» отдел...",chatgpt-4o-latest,1
9,Видали новые цены в магазинах? Зато правительс...,chatgpt-4o-latest,1


In [ ]:
print(dfAI['texts'][4])

Россия всегда будет впереди! Мы – нация, способная на великие свершения, несмотря на трудности и давление извне. Поддерживаем нашего лидера, Владимира Путина, и его курс на укрепление могущества нашей Родины. Вместе мы создаем будущее для нас и наших детей!


In [ ]:
# Re-using the same function for calculating the word counts in the generated texts.
def word_count(text):
    '''
    Tokenizes the text into words, excludes punctuation but retains the numbers, and counts word tokens.
    Returns the word tokens.
    '''

    tokens = word_tokenize(text)
    word_tokens = [word for word in tokens if word.isalnum()]

    return len(word_tokens)

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
tqdm.pandas()

dfAI['word_counts'] = dfAI['texts'].progress_apply(word_count)

100%|██████████| 500/500 [00:00<00:00, 2252.81it/s]


In [ ]:
# On average the texts are a bit shorter than the inteded 45 words goal. However, it is still comparable.
dfAI['word_counts'].describe()

,word_counts
count,500.000000
mean,33.510000
std,6.026154
min,7.000000
25%,33.000000
50%,35.000000
75%,37.000000
max,42.000000


In [ ]:
# Save the output to file.
dfAI.to_csv('data/original/ai/ai_social.csv', index=False, encoding='utf-8')

!!! Quickly fixing the columns names to follow a unified format with other datasets since we forgot to do in the previous steps when creating the dataset.

In [ ]:
df2 = pd.read_csv('data/original/ai/ai_social.csv')
df2.head()

,texts,source,word_counts,genre,class
0,"> Снова в новостях: ""свои интересы защищаем"". ...",chatgpt-4o-latest,34,social,1
1,Да кто вообще верит в эти сказки про «светлое ...,chatgpt-4o-latest,38,social,1
2,"Господа, думайте прежде, чем спорить про полит...",chatgpt-4o-latest,36,social,1
3,"Ну вот, а говорили, что отзывов не будет. Оказ...",chatgpt-4o-latest,25,social,1
4,"Россия всегда будет впереди! Мы – нация, спосо...",chatgpt-4o-latest,38,social,1


In [ ]:
df2 = df2.rename(columns={"author": "source", "ai": "class"})
df2["genre"] = "social"
df2 = df2[["texts", "source", "word_counts", "genre", "class"]]

In [ ]:
df2.head()

,texts,source,word_counts,genre,class
0,"> Снова в новостях: ""свои интересы защищаем"". ...",chatgpt-4o-latest,34,social,1
1,Да кто вообще верит в эти сказки про «светлое ...,chatgpt-4o-latest,38,social,1
2,"Господа, думайте прежде, чем спорить про полит...",chatgpt-4o-latest,36,social,1
3,"Ну вот, а говорили, что отзывов не будет. Оказ...",chatgpt-4o-latest,25,social,1
4,"Россия всегда будет впереди! Мы – нация, спосо...",chatgpt-4o-latest,38,social,1


In [ ]:
# Now re-write the file.
df2.to_csv('data/original/ai/ai_social.csv', index=False, encoding='utf-8')